## Alert API

This API provides the possibility to create an alert for jobs, feature store and project.

## Prerequisite

Global channels need to be configured by an administrator to be able to create alerts. 

![Configure Alerts](../images/alert_channels.png)

For this tutorial we need email and slack channels configured. See [Configure Alerts](https://docs.hopsworks.ai/latest/setup_installation/admin/alert/#configure-alerts) on how to configure global channels

## Scope

* Create receivers
* Create an alert for a project
* Create an alert for a job
* Triggering an alert
*  

In [1]:
import hopsworks

## Connect to the cluster

In [2]:
# Connect to your cluster, to be used running inside Jupyter or jobs inside the cluster.
project = hopsworks.login()
# Uncomment when connecting to the cluster from an external environment.
# project = hopsworks.login(host="hopsworks.ai.local", api_key_file='api_key')

2025-07-01 07:29:35,561 INFO: Python Engine initialized.

Logged in to project, explore it here https://hopsworks.ai.local/p/120


## Create receivers

In [3]:
alerts_api = project.get_alerts_api()

In [4]:
# Create email receiver
alerts_api.create_alert_receiver(
    name="email", email_configs=[{"to": "email1@mail.com"}]
)
# Create slack receiver
alerts_api.create_alert_receiver(
    name="slack", slack_configs=[{"channel": "#general"}]
)

2025-07-01 07:29:36,582 INFO: Waiting for receiver to be created.
2025-07-01 07:30:47,096 INFO: Receiver created.
2025-07-01 07:30:47,266 INFO: Waiting for receiver to be created.
2025-07-01 07:32:12,993 INFO: Receiver created.


AlertReceiver('project2__slack', [SlackConfig('#general')])

In [5]:
# Get all receivers
receivers = alerts_api.get_alert_receivers()
receivers

[AlertReceiver('project2__email', [EmailConfig('email1@mail.com')]),
 AlertReceiver('project2__slack', [SlackConfig('#general')])]

In [6]:
# Get a specific receiver
email_receiver = alerts_api.get_alert_receiver("email")
email_receiver

AlertReceiver('project2__email', [EmailConfig('email1@mail.com')])

In [7]:
# Get a specific receiver with full name
slack_receiver = alerts_api.get_alert_receiver(f"{project.name}__slack")
slack_receiver

AlertReceiver('project2__slack', [SlackConfig('#general')])

## Create an alert for a project

Alerts created within a project will be triggered by any job or data validation within that project.

In [8]:
# Create an alert that will be triggered by any job within the current project
alert = project.create_job_alert(receiver="slack", status="job_failed", severity="warning")

## Create an alert for a job

Create an alert that will trigger for a specific job

In [ ]:
dataset_api = project.get_dataset_api()
jobs_api = project.get_jobs_api()

In [ ]:
JOB_NAME = "my_alert_job"
with open(f"/tmp/{JOB_NAME}_{project.name}.py", "w") as f:
    f.write("""
import argparse
import logging
from time import sleep

parser = argparse.ArgumentParser(description ='Process some args.')
parser.add_argument('--arg1', type=str)
args = parser.parse_args()
sleep(120)

if args.arg1 == "fail":
    print("Job failed") # this will be in the stdout which is needed by wait_until_finished
    raise Exception("Failing job")
print("Job finished successfully")
logging.error("No Error occurred. But we need this because we wait for the stderr in wait_until_finished!")
    """)

uploaded_file_path = dataset_api.upload(
    f"/tmp/{JOB_NAME}_{project.name}.py", "Resources", overwrite=True
)

Uploading /tmp/my_alert_job_project2.py: 0.000%|          | 0/501 elapsed<00:00 remaining<?

In [ ]:
python_config = jobs_api.get_configuration("PYTHON")
python_config["appPath"] = uploaded_file_path

In [ ]:
my_job = jobs_api.create_job(JOB_NAME, python_config)

Job created successfully, explore it at https://hopsworks.ai.local/p/120/jobs/named/my_alert_job


In [ ]:
# Create an alert that will be triggered by this job
my_job.create_alert(receiver="slack", status="finished", severity="info")

JobAlert(1, 'my_alert_job', 'FINISHED', 'INFO', 'project2__slack', '2025-07-01T07:32:15.000Z', None)

In [ ]:
execution = my_job.run(args="--arg1 fail", await_termination=True)

Launching job: my_alert_job
Job started successfully, you can follow the progress at 
https://hopsworks.ai.local/p/120/jobs/named/my_alert_job/executions
2025-07-01 07:32:18,798 INFO: Waiting for execution to finish. Current state: INITIALIZING
2025-07-01 07:32:21,850 INFO: Waiting for execution to finish. Current state: RUNNING
2025-07-01 07:34:32,912 INFO: Waiting for execution to finish. Current state: FAILED
2025-07-01 07:34:33,061 INFO: Waiting for log aggregation to finish.
2025-07-01 07:34:33,062 ERROR: Execution failed with status: FAILED. See the logs for more information.


In [ ]:
# Check if the project alert was triggered.
triggered_alert = alerts_api.get_triggered_alerts()
triggered_alert

[TriggeredAlert({'alertname': 'JobExecution', 'execution_id': '23', 'job': 'my_alert_job', 'project': 'project2', 'severity': 'warning', 'status': 'Failed', 'type': 'project-alert'}, {'description': 'Job=my_alert_job with executionId=23 failed', 'summary': 'Job Failed', 'title': 'my_alert_job'}, [{'name': 'project2__slack'}], AlertStatus('active', [], []), '50d8f75d11f23f57', '2025-07-01T07:34:30.664Z', '2025-07-01T07:39:30.664Z', '2025-07-01T07:34:30.664Z', None)]

In [ ]:
execution = my_job.run(args="--arg1 success", await_termination=True)

Launching job: my_alert_job
Job started successfully, you can follow the progress at 
https://hopsworks.ai.local/p/120/jobs/named/my_alert_job/executions
2025-07-01 07:34:36,475 INFO: Waiting for execution to finish. Current state: RUNNING
2025-07-01 07:36:47,608 INFO: Waiting for execution to finish. Current state: FINISHED
2025-07-01 07:36:47,759 INFO: Waiting for log aggregation to finish.
2025-07-01 07:36:47,760 INFO: Execution finished successfully.


In [ ]:
# Check if the job alert was triggered.
triggered_alert = alerts_api.get_triggered_alerts()
triggered_alert

[TriggeredAlert({'alertname': 'JobExecution', 'execution_id': '23', 'job': 'my_alert_job', 'project': 'project2', 'severity': 'warning', 'status': 'Failed', 'type': 'project-alert'}, {'description': 'Job=my_alert_job with executionId=23 failed', 'summary': 'Job Failed', 'title': 'my_alert_job'}, [{'name': 'project2__slack'}], AlertStatus('active', [], []), '50d8f75d11f23f57', '2025-07-01T07:34:30.664Z', '2025-07-01T07:39:30.664Z', '2025-07-01T07:34:30.664Z', None),
 TriggeredAlert({'alertname': 'JobExecution', 'execution_id': '24', 'job': 'my_alert_job', 'project': 'project2', 'severity': 'info', 'status': 'Finished', 'type': 'project-alert'}, {'description': 'Job=my_alert_job with executionId=24 finished', 'summary': 'Job Finished', 'title': 'my_alert_job'}, [{'name': 'project2__slack'}], AlertStatus('active', [], []), 'c1eb36403d29996f', '2025-07-01T07:36:45.658Z', '2025-07-01T07:41:45.658Z', '2025-07-01T07:36:45.658Z', None)]

If you have configured slack correctly you should get two alerts.

![Slack alert](../images/job_alerts.png)

## Triggering an alert

You can also trigger an alert directly from code.

In [18]:
alerts_api.trigger_alert(receiver_name="slack", title="Test alert api", summary="Alert summary", description="Alert description", severity="info", status="script_finished", name="AlertApi")

2025-07-01 07:36:48,009 INFO: Waiting for route to be created.
2025-07-01 07:36:58,083 INFO: Route created.


In [19]:
alerts_api.get_triggered_alerts()

[TriggeredAlert({'alertname': 'JobExecution', 'execution_id': '23', 'job': 'my_alert_job', 'project': 'project2', 'severity': 'warning', 'status': 'Failed', 'type': 'project-alert'}, {'description': 'Job=my_alert_job with executionId=23 failed', 'summary': 'Job Failed', 'title': 'my_alert_job'}, [{'name': 'project2__slack'}], AlertStatus('active', [], []), '50d8f75d11f23f57', '2025-07-01T07:34:30.664Z', '2025-07-01T07:39:30.664Z', '2025-07-01T07:34:30.664Z', None),
 TriggeredAlert({'alertname': 'AlertApi', 'api': 'alerts-api', 'project': 'project2', 'severity': 'info', 'status': 'script_finished', 'type': 'project-alert'}, {'description': 'Alert description', 'summary': 'Alert summary', 'title': 'Test alert api'}, [{'name': 'project2__slack'}], AlertStatus('active', [], []), '9531696f463138f9', '2025-07-01T07:36:58.114Z', '2025-07-01T07:41:58.114Z', '2025-07-01T07:36:58.114Z', None),
 TriggeredAlert({'alertname': 'JobExecution', 'execution_id': '24', 'job': 'my_alert_job', 'project': '

![Triggering an alert](../images/script_alert.png)